In [193]:
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as npr

## European Option price

In [194]:
def norm_cdf(x):
    """ Approximation of the normal distribution function with an error less than 7.5*10^-8 """
    assert x >= 0 or x < 0
    coefficients = [0.2316419, 0.319381530, -0.356563782, 1.781477937, -1.821255978, 1.33027442]
    if x >= 0:
        t = 1 / (1 + coefficients[0] * x)
        terms = [coefficients[i] * t**i for i in range(1, 6)]
        approx = 1 - (1 / np.sqrt(2 * np.pi)) * np.exp(-x**2 / 2) * sum(terms)
        return approx
    else:
        return 1 - norm_cdf(-x)

In [195]:
def call_eur_bs(r, sig, T, S0, K):
    d = (np.log(S0 / K) + r * T + 0.5 * sig**2 * T) / (sig * np.sqrt(T))
    return (
        S0 * np.exp(r * T) * norm_cdf(sig * np.sqrt(T) - d)
        - K * norm_cdf(-d)
    )

## Turnbull & Wakeman approximation

Turnbull & Wakeman have provided an approximation of the price of an asian option, as a european option with different parameters.

In [196]:
def tnw_coeffs_con(r, sig, T):
    """ Calculates the r_a and sigma_a coefficients from the Turnbull & Wakeman approximation for an asian option """
    M1 = (np.exp(r*T)-1)/(r*T)
    M2 = ( 
        (2*np.exp((2*r+sig**2)*T)) / ((r+sig**2) * (2*r+sig**2) * T**2)
        + (2/(r*T**2)) * (1/(2*r+sig**2) - np.exp(r*T) / (r+sig**2))
    )

    r_a = np.log(M1) / T 
    sig_a = np.sqrt(-2*r_a + np.log(M2)/T)
    return (r_a, sig_a)

In [197]:
def tnw_coeffs_dis(r, sig, T, num_subdivs):
    """ Calculates the r_a and sigma_a coefficients from the Turnbull & Wakeman approximation for an asian option """
    dt = T/num_subdivs
    N = num_subdivs
    t0 = np.exp((2*r+sig**2)*dt)
    t1 = np.exp(r*dt)
    M1 = (1 / N)*( (t1*(1-np.exp(r*T))) / (1-t1) )
    M2 = (
        (1/(N**2))*( (t0 * (1-np.exp((2*r+sig**2)*T)) ) / (1-t0) )
        + (1/(N**2))*(2*t1/(1-t1))*( 
        t0 * ( (1-np.exp((2*r+sig**2)*(N-1)*dt)) / (1-t0) )
        - np.exp(((N+1)*r+sig**2)*dt) * ( (1-np.exp((r+sig**2)*(N-1)*dt)) / (1-np.exp((r+sig**2)*dt)) )
    ))

    r_A = np.log(M1) / T
    sig_a = np.sqrt(np.log(M2)/T - 2 * r_A)

    return (r_A, sig_a)

In [198]:
def call_asian_bs_tnw(r, sig, T, S0, K, num_subdivs):
    """ Calculates price of an asian option under the Turnbull & Wakeman approximation and the Black-Scholes model """
    r_a, sig_a = tnw_coeffs_dis(r, sig, T, num_subdivs)
    return np.exp(-r*T) * call_eur_bs(r_a, sig_a, T, K, S0)

## Monte-Carlo approximation

In [199]:
def price_paths_bs(r, sig, T, S0, num_paths, num_subdivs):
    step = (T/num_subdivs)
    log_diffs = (r-sig**2/2) * step + sig*np.sqrt(step)*npr.normal(0, 1, size=(num_paths, num_subdivs))
    log_diffs = np.concatenate((np.log(S0)*np.ones((num_paths, 1)), log_diffs), axis=1)
    log_paths = np.cumsum(log_diffs, axis=1)
    price_paths = np.exp(log_paths)

    return price_paths

In [200]:
def call_asian_bs_mc(r, sig, T, S0, K, num_paths, num_subdivs):
    price_paths = price_paths_bs(r, sig, T, S0, num_paths, num_subdivs)

    payoffs_asian = np.exp(-r*T)*np.maximum(np.sum(price_paths, axis=1)/num_subdivs - K, 0)
    var = payoffs_asian.var(ddof=1) / num_paths

    return payoffs_asian.mean(), var

In [201]:
def ctrl_coeffs(r, sig, dt, n):
    sig_e = sig*np.exp( ((n+1)*(2*n+1)) / (6*dt*n**2) )
    r_e = ((n+1)*(2*n+1)*sig**2) / (12*dt*n**2) + (r-0.5*sig**2)*(n+1)/(2*n)

    return (r_e, sig_e)

In [202]:
def call_asian_bs_mc_ctrl(r, sig, T, S0, K, num_paths, num_subdivs):
    """ Calculate a monte-carlo simulation of the asian price with a lower variance """
    price_paths = price_paths_bs(r, sig, T, S0, num_paths, num_subdivs)

    payoffs_asian = np.exp(-r*T)*np.maximum(np.sum(price_paths, axis=1)/num_subdivs - K, 0)
    control_samples = np.exp( (1/num_subdivs)*np.log(price_paths).sum(axis=1) )

    dt = T/num_subdivs
    r_e, sig_e = ctrl_coeffs(r, sig, dt, num_subdivs)

    control_mean = np.exp(-r*T)*call_eur_bs(r_e, sig_e, T, S0, K)
    res_sample = payoffs_asian + control_samples - control_mean

    var = res_sample.var(ddof=1) / num_paths
    return (res_sample.mean(), var)
    

## Tests

In [227]:
S0 = 100   # Spot
K = 100    # Strike
sig = 0.3  # Vol.
T = 6      # Expiration
r = 0.02   # Risk-free rate
N = 1000   # Number of simulations (monte-carlo)
n = 30     # Path subdivisions

tnw_approx = call_asian_bs_tnw(r, sig, T, S0, K, num_subdivs=n)
mc_approx, var = call_asian_bs_mc(r, sig, T, S0, K, num_paths=N, num_subdivs=n)
mc_approx_c, var_c = call_asian_bs_mc_ctrl(r, sig, T, S0, K, num_paths=N, num_subdivs=n)

print(f"Approximation Monte-Carlo  : {mc_approx:.3f} (var={var:.3f})")
print(f"Variable de contrôle       : {mc_approx_c:.3f} (var={var_c:.3f})")
print(f"Approximation T&W discrète : {tnw_approx:.3f}")

Approximation Monte-Carlo  : 20.740 (var=1.358)
Variable de contrôle       : 13.627 (var=8.183)
Approximation T&W discrète : 18.721
